<a href="https://colab.research.google.com/github/AndrewB01/JSON-XML-Tabular-transformation/blob/main/json_xml_tabular_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd

def extract_headers_and_row(data, parerent_key = " ", result = None):
    if result is None:
        result = {}
    if isinstance(data, dict):
        for key, value in data.items():
            new_key = f"{parerent_key}.{key}" if parerent_key else key
            extract_headers_and_row(value, new_key, result)
    elif isinstance(data, list):
        if data and isinstance(data[0], dict):
            return [extract_headers_and_row(item, "", {}) for item in data]
        else:
            result[parerent_key] = data
    else:
        result[parerent_key] = data

    return result

def find_data_section(response_json):
    def recursive_search(data):
        if isinstance (data, dict):
            for key, value in data.items():
                if isinstance(value, list):
                    return value
                found = recursive_search(value)
                if found:
                    return found
        elif isinstance (data, list):
            for item in data:
                found = recursive_search(item)
                if found:
                    return found
        return None
    return recursive_search(response_json) or []


def parse_soap_json(response_json):
    data_section = find_data_section(response_json)
    print("Extracted Data Section:", data_section)

    if not data_section:
        raise ValueError("No valida data section founs in SOAP response.")
    extracted_data = extract_headers_and_row(data_section)
    return extracted_data if isinstance(extracted_data, list) else [extracted_data]

def load_local_json(file_path):
    with open(file_path, "r", encoding="utf-8-sig") as file:
        return json.load(file)

file_path = "soap_response.json" # enter the file name plus the file extension type
soap_response = load_local_json(file_path)
flattened_data = parse_soap_json(soap_response)
df = pd.json_normalize(flattened_data)
df.to_csv("output_tabular.csv", index=False)
print ("tranforamtion compelted. Data save to output_tabular.csv")

Extracted Data Section: [{'b:CompanyCode': 'C001', 'b:EmployeeNumber': 372, 'b:FirstName': 'Ninon', 'b:LastName': 'LaRoche', 'b:People': {'b:Person': {'b:AlternateEmailAddress': {'i:@nil': True}, 'b:EmailAddress': 'nlaroche@logistec.com', 'b:EmployeeIdentifier': {'b:CompanyCode': 'C001', 'b:EmployeeNumber': 372, 'i:@type': 'b:EmployeeNumberIdentifier'}, 'b:FirstName': 'Ninon', 'b:FormerLastName': {'i:@nil': True}, 'b:LastName': 'LaRoche', 'b:MiddleName': {'i:@nil': True}, 'b:PreferredFirstName': 'Ninon', 'b:Prefix': {'i:@nil': True}, 'b:SSN': {'i:@nil': True}, 'b:SelfServiceProperties': {'i:@nil': True}, 'b:Suffix': {'i:@nil': True}, 'b:SuppressSSN': False}}}, {'b:CompanyCode': 'C001', 'b:EmployeeNumber': 151, 'b:FirstName': 'Geneviève', 'b:LastName': 'Gauthier', 'b:People': {'b:Person': {'b:AlternateEmailAddress': {'i:@nil': True}, 'b:EmailAddress': 'ggauthie@logistec.com', 'b:EmployeeIdentifier': {'b:CompanyCode': 'C001', 'b:EmployeeNumber': 151, 'i:@type': 'b:EmployeeNumberIdentifie